In [1]:
class state:
    def __init__(self, comprob_a, comprob_b):
        self.prob_a = comprob_a
        self.prob_b = comprob_b
    def modify(self, prob_0, prob_1):
        self.prob_a = prob_0
        self.prob_b = prob_1
    def __str__(self):
        out = "Probability: \n"
        out += "Prob of 0: "
        out += str((abs(self.prob_a))**2)
        out += '\n'
        out += "Prob of 1: "
        out += str((abs(self.prob_b))**2)
        out += '\n'
        out += self.qubit()
        out += '\n-------\n'
        return out
    def __eq__(self, other):
        if self.prob_a == other.prob_a and self.prob_b==other.prob_b:
            return True
        else:
            return False
    def qubit(self):
        out = ''
        out += 'Qubit: '
        out += str(self.prob_a) if (self.prob_a != 0) else ''
        out += '|0>' if (self.prob_a!=0) else ''
        out += '+' if ((self.prob_a!=0) and (self.prob_b!=0)) else ''
        out += str(self.prob_b) if (self.prob_b != 0) else ''
        out += '|1>' if (self.prob_b!=0) else ''
        return out

In [2]:
# Defining the state of v Gate
state_1 = state(1, 0) # define the |0>
state_2 = state((1/(0+2j)**0.5), (1j/(0+2j)**0.5))
state_3 = state(0, 1) # define the |1>
state_4 = state( (1j/(0+2j)**0.5), (1/(0+2j)**0.5))
state_arr = [state_1, state_2, state_3, state_4]

In [3]:
def vGate(cont,out):
    """ V Gate act as state machine """
    if cont==state_1:
        # No change to out
        return out
    elif cont==state_3:
        # change out state to next state
        return state_arr[(state_arr.index(out)+1)%4]

In [4]:
def vGateT(cont, out):
    """ Defining the transpose of vGate """
    if cont==state_1:
        # No change to out
        return out
    elif cont==state_3:
        # change out state to prev state
        return state_arr[(state_arr.index(out)-1)%4]

In [5]:
def cNot(cont, out):
    """ This function implement quantum cNot"""
    if cont==state_1:
        # No change to out
        return out
    elif cont==state_3:
        # change the state |0> to |1> and |1> to |0>
        if out==state_1:
            return state_3
        elif out==state_3:
            return state_1
        else:
            print("Not handled yet")

In [6]:
def ppg(a, b, s = state_1):
    """ This function simulated the working of Product pair generator (implement quantum computing) """
    s = vGate(a, s)
    a = cNot(b, a)
    s = vGate(b, s)
    s = vGateT(a, s)
    a = cNot(b, a)
    return a, b, s

In [7]:
def qfa(a, b, c, s=state_1):
    """This function implement the quantum full addition"""
    temp = (a,b,c)
    s = vGate(b, s)
    s = vGate(a, s)
    b = cNot(a, b)
    s = vGate(c, s)
    c = cNot(b, c)
    s = vGateT(c,s)
    return temp[0], temp[1], temp[2], c, s

In [8]:
def qfs(a, b, c, s=state_1):
    """This function implement the quantum full addition"""
    temp = (a,b,c)
    s = vGate(c, s)
    c = cNot(b, c)
    s = vGate(b, s)
    c = cNot(a, c)
    s = vGateT(a, s)
    s = vGate(c, s)
    return temp[0], temp[1], temp[2], c, s

In [9]:
def addition_n(a, b, n = 4):
    """ This function calculate the sum of two n-qubit number"""
    carry = [state_1]*(2*n)
    res = [state_1]*(2*n)
    for i in range(n):
        _, __, ___, res[i], carry[i+1] = qfa(a[i], b[i], carry[i])
    res[n] = carry[n]
    return res

In [10]:
def subtractor_n(a, b, n = 4):
    """ This function calculate the difference to two n-qubit number """
    carry = [state_1]*(n+1)
    diff = [state_1]*n
    for i in range(n):
        _, __, ___, diff[i], carry[i+1] = qfs(a[i], b[i], carry[i])
    return diff

In [11]:
# Implementing quantum circuit of grade school multiplication algorithm
def quantum_multiplication4X4(multiplier, multiplicand):
    """The function multiplied two quantum number"""
    x0y0 = ppg(multiplier[0], multiplicand[0])[-1]
    x0y1 = ppg(multiplier[0], multiplicand[1])[-1]
    x0y2 = ppg(multiplier[0], multiplicand[2])[-1]
    x0y3 = ppg(multiplier[0], multiplicand[3])[-1]
    x1y0 = ppg(multiplier[1], multiplicand[0])[-1]
    x1y1 = ppg(multiplier[1], multiplicand[1])[-1]
    x1y2 = ppg(multiplier[1], multiplicand[2])[-1]
    x1y3 = ppg(multiplier[1], multiplicand[3])[-1]
    x2y0 = ppg(multiplier[2], multiplicand[0])[-1]
    x2y1 = ppg(multiplier[2], multiplicand[1])[-1]
    x2y2 = ppg(multiplier[2], multiplicand[2])[-1]
    x2y3 = ppg(multiplier[2], multiplicand[3])[-1]
    x3y0 = ppg(multiplier[3], multiplicand[0])[-1]
    x3y1 = ppg(multiplier[3], multiplicand[1])[-1]
    x3y2 = ppg(multiplier[3], multiplicand[2])[-1]
    x3y3 = ppg(multiplier[3], multiplicand[3])[-1]
    pro = [state_1]*8
    c = state_1
    pro[0], c = qfa(x0y0, state_1, state_1)[3:]
    pro[1], c = qfa(x1y0, x0y1, c)[3:]
    pro[2],c = qfa(x2y0, x1y1, c)[3:]
    pro[2],c = qfa(pro[2],x0y2,c)[3:]
    pro[3],c = qfa(x3y0, x2y1, c)[3:]
    pro[3],c = qfa(pro[3],x1y2, c)[3:]
    pro[3],c = qfa(pro[3],x0y3, c)[3:]
    pro[4],c = qfa(x3y1, x2y2, c)[3:]
    pro[4],c = qfa(pro[4],x1y3,c)[3:]
    pro[5],c = qfa(x3y2, x2y3, c)[3:]
    pro[6],c = qfa(x3y3, state_1, c)[3:]
    pro[7] = c
    return pro

In [12]:
def quantum_multiplication8x8(multiplier, multiplicand):
    """The function multiplies two 8-bit quantum numbers and returns a 16-bit quantum result."""
    # Initialize a list to store the partial products
    partial_products = [[state_1] * 8 for _ in range(8)]
    
    # Calculate the partial products
    for i in range(8):
        for j in range(8):
            partial_products[i][j] = ppg(multiplier[i], multiplicand[j])[-1]
    
    # Initialize the product array with 16 bits
    pro = [state_1] * 16
    
    # Initialize carry bits
    c = [state_1] * 15
    
    # Sum the partial products using quantum full adders
    for i in range(8):
        for j in range(8):
            if i + j == 0:
                pro[0], c[0] = qfa(partial_products[0][0], state_1, state_1)[3:]
            else:
                idx = i + j
                pro[idx], c[idx] = qfa(partial_products[i][j], pro[idx], c[idx-1])[3:]
    
    # Handle the final carry bits
    for k in range(8, 15):
        pro[k+1], c[k] = qfa(state_1, pro[k+1], c[k])[3:]
    
    return pro

In [13]:
a = [state_3, state_1, state_3, state_3] # 1101
b = [state_1, state_1, state_3, state_1] # 0100
p = quantum_multiplication4X4(a, b) # 0110100

In [14]:
for (i,x) in enumerate(p):
    print(i, x.qubit())

0 Qubit: 1|0>
1 Qubit: 1|0>
2 Qubit: 1|1>
3 Qubit: 1|0>
4 Qubit: 1|1>
5 Qubit: 1|1>
6 Qubit: 1|0>
7 Qubit: 1|0>


In [15]:
def get_result(quantum):
    """This function convert quantum result to numerical value"""
    num = '0b'+''.join(list(map(lambda x: '0' if x==state_1 else '1', quantum))[::-1])
    print(num)
    print(int(num,2))

In [16]:
get_result(a)
get_result(b)
get_result(p)

0b1101
13
0b0100
4
0b00110100
52


In [17]:
def mapper(num1, num2, length=4):
    """This function take two number and return its quatum state"""
    bin1 = bin(num1)[2:]
    bin2 = bin(num2)[2:]
    t = length
    bin1 = '0'*(t-len(bin1))+bin1
    bin2 = '0'*(t-len(bin2))+bin2
    bin1 = list(map(lambda x: state_1 if x=='0' else state_3, bin1))
    bin2 = list(map(lambda x: state_1 if x=='0' else state_3, bin2))
    bin1 = bin1[::-1]
    bin2 = bin2[::-1]
    return bin1, bin2

In [18]:
a, b = mapper(13, 4)
get_result(a)
get_result(b)

0b1101
13
0b0100
4


In [19]:
p = quantum_multiplication4X4(a, b)
get_result(p)

0b00110100
52


In [20]:
a, b = mapper(13, 4, 8)
p = quantum_multiplication8x8(a, b)

In [21]:
get_result(p)

0b0000000000110100
52


In [22]:
arr = [1, 2, 3, 4]
arr[:2]

[1, 2]

In [23]:
a, b = mapper(13, 4)
c = addition_n(a, b)
get_result(c)

0b00010001
17


In [24]:
get_result(a)
get_result(b)

0b1101
13
0b0100
4


In [25]:
c = subtractor_n(a, b)
get_result(c)

0b1001
9


In [71]:
def karatsuba(num1, num2):

    Y1 = num1[:4]
    X1 = num1[4:]
    get_result(X1)
    get_result(Y1)
    
    Y2 = num2[:4]
    X2 = num2[4:]
    get_result(X2)
    get_result(Y2)
    
    U = quantum_multiplication4X4(X1, X2)
    V = quantum_multiplication4X4(Y1, Y2)
    
    W1 = addition_n(X1, Y1)
    W2 = addition_n(X2, Y2)
    
    W = quantum_multiplication8x8(W1, W2)
    temp = addition_n(U, V, 8)
    Z = subtractor_n(W, temp, 16)
    
    U = [state_1]*8+U
    Z = [state_1]*4+Z[:-4]
    V = V+[state_1]*8

    res = [state_1] * 16
    carry = [state_1]*17
    for i in range(16):
        _, __, ___, res[i], carry[i+1] = qfa(U[i], V[i], Z[i])
    for i in range(16):
        _, __, ___, res[i], carry[i+1] = qfa(res[i], carry[i], state_1)

    return res
    

In [83]:
a, b = mapper(10, 8, 8)

In [84]:
get_result(a)
get_result(b)

0b00001010
10
0b00001000
8


In [85]:
c = karatsuba(a, b)

0b0000
0
0b1010
10
0b0000
0
0b1000
8
0b00000000
0
0b01010000
80
0b00001010
10
0b00001000
8
0b0000000001010000
80
0b0000000001010000
80
0b0000000000000000
0
0b00000000
0
0b01010000
80
0b0000000001010000
80


In [86]:
get_result(c)

0b0000000001010000
80


In [64]:
x, y=mapper(400, 0)

In [65]:
get_result(x)

0b110010000
400


In [78]:
a, b = mapper(3, 3, 8)
c = quantum_multiplication8x8(a, b)
get_result(a)
get_result(b)
get_result(c)

0b00000011
3
0b00000011
3
0b0000000000001101
13
